In [1]:
#https://www.dummies.com/programming/python/plotting-a-sound-file-in-ipython/
import matplotlib.pyplot as plt
#from scipy.io import wavfile as wav
from scipy.fftpack import fft
import numpy as np
from gurobipy import *
from IPython.display import Audio
import os
import glob
import untangle

import xml.etree.ElementTree as et
import pandas as pd
import re
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
import soundfile

In [27]:
conda list

# packages in environment at /Users/michellengu/anaconda3/envs/AMt_env:
#
# Name                    Version                   Build  Channel
appnope                   0.1.0                    py37_0  
attrs                     19.3.0                     py_0  
backcall                  0.1.0                    py37_0  
blas                      1.0                         mkl  
bleach                    3.1.0                    py37_0  
ca-certificates           2019.10.16                    0  
certifi                   2019.9.11                py37_0  
cffi                      1.13.2                   pypi_0    pypi
cycler                    0.10.0                   py37_0  
dbus                      1.13.12              h90a0687_0  
decorator                 4.4.1                      py_0  
defusedxml                0.6.0                      py_0  
entrypoints               0.3                      py37_0  
expat                     2.2.6                h0a44026_0  
freetype     

# Functions

In [2]:
# Function for transforming a signal with fourier
def fourier_trans(data, rate, begin_time, end_time): 
    # initialising variables
    tp_count = len(data)
    values = np.arange(int(tp_count/2))
    time_period = tp_count/rate
    frequencies = values/time_period
    
    #at what intervals time points are sampled
    sampling_interval = 1/rate;
    
    #time points
    time = np.arange(begin_time, end_time, sampling_interval);
    
    #Frequency domain representation
    fourier_transform = fft(data)/tp_count #normalized the amplitude
    fourier_transform = fourier_transform[range(int(len(data)/2))] #exclude sampling frequency   
    return frequencies, abs(fourier_transform)

# TODO: Funktion überarbeiten damit man es auch versteht!
def new_fft(data, rate):
    N = len(data)

    # sample spacing
    T = 1.0 / rate
    x = np.linspace(0.0, N*T, N)

    yf = fft(data)
    freq = np.linspace(0.0, 1.0/(2.0*T), N//2)
    ampl = 2.0/N * np.abs(yf[0:N//2])
    return freq, ampl

# Frequency domain representation
def plot_freq_domain(freq, fourier, name):
    plt.figure(figsize=(20,5))
    plt.plot(freq, fourier)
    plt.xlabel('Frequency in Hz')
    plt.ylabel('Amplitude')
    plt.xlim(0, 1400)
    plt.title(name)
    plt.show()

# TODO: Warnung herausgeben wenn bei der Teilung der Samplerate eine Fliesskommazahl entsteht
def easy_downsampling(data, rate, sampl_fac):
    ds_data = []
    for i in range(len(data)):
        if (i%sampl_fac == 0):
            ds_data.append(data[i])
    ds_rate = int(rate/sampl_fac)
    return ds_data, ds_rate

#ToDo: startwert von 0 nicht möglich überarbeiten!!
def snip_wav(data, rate, start_sec, end_sec):
    start_point = int((rate * start_sec)-1)
    end_point = int((rate * end_sec)-1)
    #print('start_point:', start_point)
    #print('end_point:', end_point)
    snip_data = data[start_point:end_point]
    #print(snip_data)  
    return snip_data, rate

def metric(target_v, approx_v):
    norm_factor = np.linalg.norm(approx_v)
    if (norm_factor != 0):
        norm_v = approx_v/norm_factor
        diff = target_v - norm_v
        diff_sum = np.sum(np.abs(diff))
        return diff_sum
    else:
        print('Der Vektor kann nicht normalisiert werden.')
        return approx_v 

# normalize vector
def norm_vec(vector):
    norm_factor = 0
    for i in range(len(vector)):
        norm_factor += pow(vector[i],2)
        norm_factor = math.sqrt(norm_factor)
    norm_factor
    
    for n in range(len(vector)):
        vector[n] = vector[n]/norm_factor
    einheit = 0

    for m in range(len(vector)):
        einheit += pow(vector[m],2)
        einheit = round(math.sqrt(einheit))
        einheit
    return vector, einheit

## Dataframe 

In [23]:
def read_xml_to_df(path, df_cols, offset_sec, duration_sec, sampl_fac):
    dataset = re.search(r'dataset.*\b', path).group(0)
    path_xml = path + "annotation"
    path_wav = path + "audio"  
    rows = []
    
    for xml_file in sorted(glob.glob(os.path.join(path_xml, '*.xml'))):
        tree = et.parse(xml_file)
        root = tree.getroot()
       
        for globalParam in root.findall('globalParameter'):
            audio_file = globalParam.find('audioFileName').text
            print(audio_file)
            
            # Read wav-file
            wav_file = path_wav + '/' + audio_file
            data, rate = soundfile.read(wav_file)

        for transcription in root.findall('transcription'):

            for event in transcription.findall('event'):
                res = []
                res.append(dataset)
                res.append(audio_file)

                for elem in df_cols[len(res):]:
                    if event is not None:
                        if event.find(elem) is not None:
                            res.append(event.find(elem).text)
                            
                            if elem == 'onsetSec':
                                onsetSec = event.find(elem).text
                                start_sec = round(offset_sec + float(onsetSec), 3)
                                end_sec = round(start_sec + duration_sec, 3)

                                # Snip and downsampling audio-file
                                data_snip, rate_snip = snip_wav(data, rate, start_sec, end_sec)
                                data_down, rate_down = easy_downsampling(data_snip, rate_snip, sampl_fac)
                                
                                # Calc FFT
                                freq, fourier = new_fft(data_down, rate_down)
                                                       
                        elif elem == 'FFT':
                            res.append(fourier)

                        elif elem == 'Freq':
                            res.append(freq)
                        else:
                            res.append(None)
                    else:
                        res.append(None)

                rows.append({df_cols[i]: res[i] for i, _ in enumerate(df_cols)})

    return pd.DataFrame(rows, columns=df_cols)

# Init Dataframe

In [24]:
# Set audio snippets and sample compression
offset_sec = 0.01
duration_sec = 0.03
sampl_fac = 4

# Create empty Dataframe
df_cols = ['dataset', 'audioFileName', 'pitch', 'onsetSec', 'FFT', 'Freq']
df_ref = pd.DataFrame(columns=df_cols)
df_test = pd.DataFrame(columns=df_cols)

# Create Dataframe with reference tones

In [25]:
%%time
# paths
path_main = "IDMT-SMT-GUITAR_V2/"
path_main_ref = path_main + "dataset1/Fender Strat Clean Neck SC/"

df_ref = read_xml_to_df(path_main_ref, df_cols, offset_sec, duration_sec, sampl_fac)
df_ref

G53-40100-1111-00001.wav
G53-41101-1111-00002.wav
G53-42102-1111-00003.wav
G53-43103-1111-00004.wav
G53-44104-1111-00005.wav
G53-45105-1111-00006.wav
G53-45200-1111-00014.wav
G53-46106-1111-00007.wav
G53-46201-1111-00015.wav
G53-47107-1111-00008.wav
G53-47202-1111-00016.wav
G53-48108-1111-00009.wav
G53-48203-1111-00017.wav
G53-49109-1111-00010.wav
G53-49204-1111-00018.wav
G53-50110-1111-00011.wav
G53-50205-1111-00019.wav
G53-50300-1111-00027.wav
G53-51111-1111-00012.wav
G53-51206-1111-00020.wav
G53-51301-1111-00028.wav
G53-52112-1111-00013.wav
G53-52207-1111-00021.wav
G53-52302-1111-00029.wav
G53-53208-1111-00022.wav
G53-53303-1111-00030.wav
G53-54209-1111-00023.wav
G53-54304-1111-00031.wav
G53-55210-1111-00024.wav
G53-55305-1111-00032.wav
G53-55400-1111-00040.wav
G53-56211-1111-00025.wav
G53-56306-1111-00033.wav
G53-56401-1111-00041.wav
G53-57212-1111-00026.wav
G53-57307-1111-00034.wav
G53-57402-1111-00042.wav
G53-58308-1111-00035.wav
G53-58403-1111-00043.wav
G53-59309-1111-00036.wav


,dataset,audioFileName,pitch,onsetSec,FFT,Freq
0,dataset1/Fender Strat Clean Neck SC,G53-40100-1111-00001.wav,40,0.2,"[0.0034458125826095165, 0.004362595487135711, ...","[0.0, 33.61280487804878, 67.22560975609755, 10..."
1,dataset1/Fender Strat Clean Neck SC,G53-41101-1111-00002.wav,41,0.2,"[0.0008083931268882175, 0.005238092712637641, ...","[0.0, 33.61280487804878, 67.22560975609755, 10..."
2,dataset1/Fender Strat Clean Neck SC,G53-42102-1111-00003.wav,42,0.2,"[0.009609625778889729, 0.011406250985680174, 0...","[0.0, 33.61280487804878, 67.22560975609755, 10..."
3,dataset1/Fender Strat Clean Neck SC,G53-43103-1111-00004.wav,43,0.2,"[0.001399567480173716, 0.0014167871838317184, ...","[0.0, 33.61280487804878, 67.22560975609755, 10..."
4,dataset1/Fender Strat Clean Neck SC,G53-44104-1111-00005.wav,44,0.2,"[0.007340630015577794, 0.009544653537432508, 0...","[0.0, 33.61280487804878, 67.22560975609755, 10..."
...,...,...,...,...,...,...
73,dataset1/Fender Strat Clean Neck SC,G53-72608-1111-00074.wav,72,0.2,"[0.002201322342333837, 0.0030289768696785827, ...","[0.0, 33.61280487804878, 67.22560975609755, 10..."
74,dataset1/Fender Strat Clean Neck SC,G53-73609-1111-00075.wav,73,0.2,"[0.0007871875590067976, 0.0006157842188591816,...","[0.0, 33.61280487804878, 67.22560975609755, 10..."
75,dataset1/Fender Strat Clean Neck SC,G53-74610-1111-00076.wav,74,0.2,"[0.0026093912268693354, 0.0029429146779266016,...","[0.0, 33.61280487804878, 67.22560975609755, 10..."
76,dataset1/Fender Strat Clean Neck SC,G53-75611-1111-00077.wav,75,0.2,"[0.0034561387721865557, 0.0023014009392835423,...","[0.0, 33.61280487804878, 67.22560975609755, 10..."


# Create Dataframe with testdatas

In [26]:
%%time
# paths testdata
# ToDO: Automatisieren #os.listdir(path_main + path_dataset)

path_testdata = [
    path_main + 'dataset1/Fender Strat Clean Neck SC/',
    path_main + 'dataset1/Fender Strat Clean Neck SC Chords/',
    path_main + 'dataset1/Ibanez Power Strat Clean Bridge HU/',
    path_main + 'dataset1/Ibanez Power Strat Clean Bridge HU Chords/',
    path_main + 'dataset1/Ibanez Power Strat Clean Bridge+Neck SC/',
    path_main + 'dataset1/Ibanez Power Strat Clean Neck HU/',
    path_main + 'dataset2/']

# dataset2:
# [Errno 2] No such file or directory: 'IDMT-SMT-GUITAR_V2/dataset2/audio/\\AR_A_fret_1-20.wav'
# in xml file den audionamen angepasst AR_A_fret_1-20.wav --> AR_A_fret_0-20.wav
# Alle 0-20 und Lick11
# Gesamthaft mussten 27 von 261 XML-Dateien überarbeitet werden

for path in path_testdata[6:]:
    df_act = read_xml_to_df(path, df_cols, offset_sec, duration_sec, sampl_fac)
    df_test = df_test.append(df_act, ignore_index=True)
df_test

\AR_A_fret_0-20.wav


RuntimeError: Error opening 'IDMT-SMT-GUITAR_V2/dataset2/audio/\\AR_A_fret_0-20.wav': System error.

In [ ]:
plot_freq_domain(df_ref.Freq[5], df_ref.FFT[5], df_ref.pitch[5])

In [ ]:
plot_freq_domain(df_ref.Freq[6], df_ref.FFT[6], df_ref.pitch[6])

In [ ]:
plot_freq_domain(df_test.Freq[87], df_test.FFT[87], df_test.audioFileName[87])

# Gurobi

In [ ]:
# Build Model
m = Model('AMt')

lambs = []
for lamb in range(len(df_ref)):
    lambs.append(m.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = 'lamb_' + str(lamb)))
m.update()

In [ ]:
'''
%%time
#Optimize Signal

deviation = 0
approxi_sgn = 0
 
for sgn in range(len(mischsignal)):
#for sgn in range(100):
    for lamb in range(len(lambs)):
        approxi_sgn += lambs[lamb] * ref_note[lamb]['fourier'][sgn]
        
    deviation += ((mischsignal[sgn] - approxi_sgn)*(mischsignal[sgn] - approxi_sgn))
    # deviation wird ausmultipliziert stattdessen die für 'mischsignal[sgn] - approxi_sgn'
    # eine Gurobi-Variable (m.addVar  inkl. constr.) verwenden
    # Überprüfen ob der Betrag der Differenz besser geeignet ist?
print('Deviation is calculated.')
'''

In [ ]:
%%time
vec_pred = []

for index, row in df_test.iterrows():
    mischsignal = row.FFT
    #print(row.audioFileName)


    # Schlaufen vertauscht
    cost_function = 0
    approxi_sgn = 0

    for lamb in range(len(lambs)):
        la = lambs[lamb]
        #print('la:', la)
        #note = ref_note[lamb]['fourier']
        note = df_ref.FFT[lamb]

        for sgn in range(len(mischsignal)):
            approxi_sgn = la * note[sgn]
            mi = mischsignal[sgn]
            cost_function += ((mi - approxi_sgn)*(mi - approxi_sgn))
    #print('Cost Function is created')

    m.params.outputflag = 0 # Infotext ausblenden
    m.setObjective(cost_function, GRB.MINIMIZE)
    m.optimize()



    approx_v = []

    for v in m.getVars():
        #print('%s: %g' % (v.varName, v.x))
        approx_v.append(v.x)

    vec_pred.append(approx_v)

In [ ]:
df_test['Vec_Pred'] = vec_pred
df_test.head()

In [ ]:
# Berechnung Vector target
def calc_target_vec(pitch):
    pitch_start = 40
    #pitch_end = 76
    pitch_end = pitch_start + 77 #TODO: muss angepasst werden
    target_vec = []
    
    if pitch_start <= pitch <= pitch_end:
        for pitch_act in range(pitch_start, pitch_end+1):
            
            if pitch_act == pitch:
                target_vec.append(1)
            else:
                target_vec.append(0)
    else:
        print('Pitch muss innerhalb der Range({} - {})!'.format(pitch_start, pitch_end))
    
    return target_vec

# Metrik berechnen

In [ ]:
# Calc all target vectors
target_pitches = df_test.pitch.array
target_vec = []

for pitch in target_pitches:
    target_vec.append(calc_target_vec(int(pitch)))

df_test['Vec_Target'] = target_vec
df_test.head()

In [ ]:
# Calc Pitch score for all events
pitch_score = []
vec_target = df_test.pitch.array

for vec in range(len(vec_pred)):
    pitch_score.append(metric(target_vec[vec], vec_pred[vec]))
    
df_test['Pitch_Score'] = pitch_score

# Calculate F1 Score

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
def plot_confusion_matrix(y_true, y_pred,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    cm = confusion_matrix(y_true, y_pred)    
    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           #xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

In [ ]:
# thresholding vec_pred
%time
vec_pred_thresh = []
for ind in range(len(df_test.Vec_Pred)):
    vec_thresh = []
    for i in range(len(df_test.Vec_Pred[ind])):
        if df_test.Vec_Pred[ind][i] > 0.9:
            vec_thresh.append(1)
        else:
            vec_thresh.append(0)
    vec_pred_thresh.append(vec_thresh)
df_test['Thresholded'] = vec_pred_thresh
df_test.head()

In [ ]:
# F1 Score with flattened vector
combi_target_vec=[]
for sublist in target_vec:
    for item in sublist:
        combi_target_vec.append(item)

combi_thresh_vec=[]
for sublist in vec_pred_thresh:
    for item in sublist:
        combi_thresh_vec.append(item)

f1_scr = f1_score(combi_target_vec, combi_thresh_vec, average='weighted')  
#plot_confusion_matrix(target_vec, vec_pred_thresh, title=f1_scr)

In [ ]:
f1_scr

In [ ]:
# F1 Score with binarized vector
true_pred = []
for ind in range(len(df_test)):
    target_vec = df_test.Vec_Target[ind]
    pred_vec = df_test.Thresholded[ind]

    if(target_vec == pred_vec):
        true_pred.append(1)
    else:
        true_pred.append(0)
true_pred

target = [1]*len(true_pred)

target
f1_scr = f1_score(target, true_pred, average='weighted')  
f1_scr

In [ ]:
#Normalize FFT
%%time
norm_fft_col = []
for vec in range(len(df_test.FFT)):
    norm_fft = []
    for i in range(len(df_test.FFT[i])):
        norm_fft.append(norm_vec(df_test.FFT[i]))
    norm_fft_col.append(norm_fft)
norm_fft_col

In [ ]:
df_test['norm_FFT'] = norm_fft_col
df_test.norm_FFT